In [4]:
!pip install pandas

^C


  Obtaining dependency information for pandas from https://files.pythonhosted.org/packages/29/d4/1244ab8edf173a10fd601f7e13b9566c1b525c4f365d6bee918e68381889/pandas-2.2.3-cp312-cp312-win_amd64.whl.metadata
  Using cached pandas-2.2.3-cp312-cp312-win_amd64.whl.metadata (19 kB)
  Obtaining dependency information for numpy>=1.26.0 from https://files.pythonhosted.org/packages/46/69/8c4f928741c2a8efa255fdc7e9097527c6dc4e4df147e3cadc5d9357ce85/numpy-2.2.4-cp312-cp312-win_amd64.whl.metadata
     ---------------------------------------- 0.0/60.8 kB ? eta -:--:--
     ------ --------------------------------- 10.2/60.8 kB ? eta -:--:--
     ------------ ------------------------- 20.5/60.8 kB 217.9 kB/s eta 0:00:01
     ------------------------------- ------ 51.2/60.8 kB 372.4 kB/s eta 0:00:01
     -------------------------------------- 60.8/60.8 kB 404.8 kB/s eta 0:00:00
  Obtaining dependency information for pytz>=2020.1 from https://files.pythonhosted.org/packages/81/c4/34e93fe5f5429d7570ec1fa


[notice] A new release of pip is available: 23.2.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv(r"C:\Users\Alka\Documents\WeatherWarden---Weather-Impact-Predictor\Backend-ML\src\Data_gathering\weather_data1.csv")
df.head()

,date,temperature,rainfall,humidity,wind_speed,pressure,visibility,weather_condition,severity,travel_disruption,road_condition,traffic_density
0,2023-01-01,13.5,13.5,71.7,4.1,1012.8,4.2,rain,severe,cancellation,wet,36
1,2023-01-02,11.7,0.6,88.4,14.7,980.7,5.8,storm,moderate,delay,flooded,49
2,2023-01-03,8.1,5.4,72.0,4.3,1004.0,10.4,rain,moderate,none,wet,54
3,2023-01-04,15.1,1.6,86.4,8.6,1012.2,13.1,rain,mild,delay,wet,38
4,2023-01-05,3.9,2.7,78.4,6.6,998.7,8.5,rain,mild,delay,wet,56


In [3]:
df.isnull().sum()

date                 0
temperature          0
rainfall             0
humidity             0
wind_speed           0
pressure             0
visibility           0
weather_condition    0
severity             0
travel_disruption    0
road_condition       0
traffic_density      0
dtype: int64

In [3]:
df.travel_disruption.value_counts()

travel_disruption
delay           532
none            511
cancellation    157
Name: count, dtype: int64

In [5]:
df['date'] = pd.to_datetime(df['date'])
df['day_of_year'] = df['date'].dt.dayofyear
df['month'] = df['date'].dt.month
df['day_of_week'] = df['date'].dt.dayofweek
df['is_weekend'] = df['day_of_week'].isin([5, 6]).astype(int)
    
df['month_sin'] = np.sin(2 * np.pi * df['month']/12)
df['month_cos'] = np.cos(2 * np.pi * df['month']/12)
df['day_sin'] = np.sin(2 * np.pi * df['day_of_year']/365)
df['day_cos'] = np.cos(2 * np.pi * df['day_of_year']/365)

In [6]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
from sklearn.preprocessing import LabelEncoder

In [7]:
features = ['month_sin', 'month_cos', 'day_sin', 'day_cos']
X = df[features]

# Encode target labels
le = LabelEncoder()
y = le.fit_transform(df['weather_condition'])

# Train/test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Model training
clf = RandomForestClassifier(random_state=42)
clf.fit(X_train, y_train)

# Prediction and Evaluation
y_pred = clf.predict(X_test)
print(classification_report(y_test, y_pred, target_names=le.classes_))

              precision    recall  f1-score   support

      cloudy       0.22      0.24      0.23        46
         fog       0.09      0.05      0.07        19
        rain       0.26      0.25      0.26        72
       storm       0.00      0.00      0.00         8
       sunny       0.40      0.45      0.42        95

    accuracy                           0.30       240
   macro avg       0.20      0.20      0.20       240
weighted avg       0.29      0.30      0.29       240



In [8]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, classification_report

# Predict on test set
y_pred = clf.predict(X_test)

# Accuracy
accuracy = accuracy_score(y_test, y_pred)

# Precision, Recall, F1 Score (macro handles class imbalance better)
precision = precision_score(y_test, y_pred, average='macro')
recall = recall_score(y_test, y_pred, average='macro')
f1 = f1_score(y_test, y_pred, average='macro')

# Confusion Matrix
cm = confusion_matrix(y_test, y_pred)

# Classification Report
report = classification_report(y_test, y_pred, target_names=le.classes_)

# Print results
print("✅ Evaluation Metrics:")
print(f"Accuracy       : {accuracy:.2f}")
print(f"Precision (macro): {precision:.2f}")
print(f"Recall (macro)   : {recall:.2f}")
print(f"F1 Score (macro) : {f1:.2f}")
print("\n📊 Confusion Matrix:")
print(cm)
print("\n📄 Classification Report:")
print(report)


✅ Evaluation Metrics:
Accuracy       : 0.30
Precision (macro): 0.20
Recall (macro)   : 0.20
F1 Score (macro) : 0.20

📊 Confusion Matrix:
[[11  3 11  0 21]
 [ 5  1  4  0  9]
 [15  4 18  2 33]
 [ 1  0  5  0  2]
 [17  3 30  2 43]]

📄 Classification Report:
              precision    recall  f1-score   support

      cloudy       0.22      0.24      0.23        46
         fog       0.09      0.05      0.07        19
        rain       0.26      0.25      0.26        72
       storm       0.00      0.00      0.00         8
       sunny       0.40      0.45      0.42        95

    accuracy                           0.30       240
   macro avg       0.20      0.20      0.20       240
weighted avg       0.29      0.30      0.29       240

